## 08. 이진화
- 임계값을 기준으로 색상을 흰색과 검은색으로 나누는 것
- 색상의 밝기 기준이기에 Grayscale에서 가능

In [1]:
import cv2
import numpy as np

BOOK = "../images/book.jpg"
PARROT_PATH = "../images/parrot.jpg"

### 8-1. 기본 이진화
- ret, binary = cv2.threshold()

In [11]:
img = cv2.imread(BOOK, cv2.IMREAD_GRAYSCALE) # 이미지를 흑백 사진으로

ret, binary = cv2.threshold(img, 190, 255, cv2.THRESH_BINARY) # 원본, 임계값, 0과 대비될 값, 옵션 - 임계값보다 크면 흰색 작으면 검은색
print(ret) # 임계값이 ret으로 반환
print(binary) # 255와 0으로 나누어진 값

cv2.imshow("img", img)
cv2.imshow("binary", binary)

cv2.waitKey(0)
cv2.destroyAllWindows()

190.0
[[255 255 255 ...   0   0   0]
 [255 255 255 ... 255 255   0]
 [255 255 255 ... 255 255 255]
 ...
 [  0   0   0 ...   0   0   0]
 [  0   0   0 ...   0   0   0]
 [  0   0   0 ...   0   0   0]]


In [19]:
# Track bar 만들기 연습
img = cv2.imread(PARROT_PATH)

name = "trackbar"
cv2.namedWindow(name) # 창을 미리 생성
trackbar_name = "Rotation"
cv2.createTrackbar(trackbar_name, name, 0, 360, lambda x: x)
while True:
    angle = cv2.getTrackbarPos(trackbar_name, name) # angle을 name의 창의 trackbar에서 설정한 값으로 함
    center = (img.shape[1] // 2, img.shape[0] // 2)
    affine = cv2.getRotationMatrix2D(center, angle, 1) # angle - trackbar의 값 반영
    dst = cv2.warpAffine(img, affine, (img.shape[1], img.shape[0])) # matrix 입력하는 부분에 affine을 입력

    cv2.imshow(name, dst) # 반영되어 변형된 창 표시. 이름은  name에서 정의된 이름 따라감

    if cv2.waitKey(1) == ord("q") : #변형에 계속 대응하기 위해 대기시간 1밀리초 부여
        break

cv2.destroyAllWindows()

### 실습 2-2. Threshold에 적용
- 트랙바 연습을 참고하여 Threshold에 트랙바 적용해보기

In [20]:
img = cv2.imread(BOOK, cv2.IMREAD_GRAYSCALE)

name = "trackbar"
cv2.namedWindow(name)
trackbar_name = "Threshold"
cv2.createTrackbar(trackbar_name, name, 0, 255, lambda x: x) # 화면에 트랙바 추가
while True :
    threshold = cv2.getTrackbarPos(trackbar_name, name) # 트랙바 값 기준으로 임계값 바꾸기
    ret, binary = cv2.threshold(img, threshold, 255, cv2.THRESH_BINARY)

    if not ret :
        break

    cv2.imshow(name, binary)

    if cv2.waitKey(1) == ord("q") :
        break

cv2.destroyAllWindows()

### 실습 2-1. 컬러 팔레트 만들기
- RGB 값을 변경할 수 있는 트랙바를 만들고
- RGB 값이 바뀜에 따라 배경색이 변하는 화면을 만들자
- 기본 색상은 검은색으로 설정
- 시간이 남는다면 트랙바의 작동을 제어할 수 있는 ON/OFF 스위치도 추가

In [8]:
img = np.zeros((400, 600, 3), dtype=np.uint8) # 검은 화면

name = "trackbar"
cv2.namedWindow(name)
trackbar1 = "Red"
trackbar2 = "Green"
trackbar3 = "Blue"
trackbar4 = "0:OFF\n1:ON"
cv2.createTrackbar(trackbar1, name, 0, 255, lambda x: x) # 화면에 트랙바 추가
cv2.createTrackbar(trackbar2, name, 0, 255, lambda x: x) # 화면에 트랙바 추가
cv2.createTrackbar(trackbar3, name, 0, 255, lambda x: x) # 화면에 트랙바 추가
cv2.createTrackbar(trackbar4, name, 0, 1, lambda x: x) # 0 일떄 off, 1일때 on

while True:
    red = cv2.getTrackbarPos(trackbar1, name)
    green = cv2.getTrackbarPos(trackbar2, name)
    blue = cv2.getTrackbarPos(trackbar3, name)
    
    on_off = cv2.getTrackbarPos(trackbar4, name)

    if on_off == 1 : # on/off 값이 1일때만
        img[:] = (blue, green, red) # RGB 값 반영
    else :
        img[:] = 0

    cv2.imshow(name, img)

    if cv2.waitKey(1) == ord("q") :
        break

cv2.destroyAllWindows()

### 8-2. 적응형 이진화
- 블록마다 다른 임계값을 적용

In [3]:
img = cv2.imread(BOOK, cv2.IMREAD_GRAYSCALE)

name = "Adaptive"
cv2.namedWindow(name)

binary = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 25, 1) # 소스, 최대값, 적응형 이진화를 할 방법

cv2.imshow(name, binary)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [5]:
img = cv2.imread(BOOK, cv2.IMREAD_GRAYSCALE)

name = "Adaptive"
cv2.namedWindow(name)


cv2.createTrackbar("block_size", name, 25, 100, lambda x: x) # 1보다 큰 홀수만 가능
cv2.createTrackbar("C", name, 1, 10, lambda x: x) # 일반적으로 양수를 사용

while True:
    block_size = cv2.getTrackbarPos("block_size", name) # 블럭 사이즈가 100이 되면 일반 threshold와 비슷하게 변함. 
    #사이즈가 작을수록, 10 미만일수록 구별되는 구간들 많아짐
    C = cv2.getTrackbarPos("C", name) # C가 커질수록 전체적으로 하얗게 변함

    if block_size <= 1: #1보다 작거나 같으면
        block_size = 3

    if block_size % 2 == 0: # 짝수면 홀수로
        block_size += 1

    binary = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, block_size, C) # 소스, 최대값, 적응형 이진화를 할 방법
    cv2.imshow(name, binary)
    if cv2.waitKey(1) == ord("q"):
        break


cv2.destroyAllWindows()

### 8-3. 오츠 알고리즘
- 최적의 Threshold를 찾는 알고리즘

In [8]:
img = cv2.imread(BOOK, cv2.IMREAD_GRAYSCALE)
ret_1, binary = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY) # threshold 값 고정으로 하는 기존방식
ret_2, otsu = cv2.threshold(img, -1, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU) # threshold 값을 자동으로 계산을 하려고 함. 임계점에 아무 값이나 넣어도 상관 없음

#print(ret_1, ret_2) # 앞은 127, 뒤는 오츠 알고리즘이 찾은 138이 나왔음

cv2.imshow("img", img)
cv2.imshow("threshold", binary)
cv2.imshow("otsu", otsu)

cv2.waitKey(0)
cv2.destroyAllWindows()